# CLASSIFICATION

In [7]:
# imports and setup
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import pandas as pd

In [12]:
# data set
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength','PetalWidth', 'Species']
SPECIES = ['Setosa', 'Vesicolor', 'Virginica']

In [13]:
# we use tf module keras to get the datasets and read them into pd df
train_path = tf.keras.utils.get_file("iris_training.csv","https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file("iris_test.csv","https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

In [14]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [15]:
# pop species col and use that as label
train_y = train.pop('Species')
test_y = test.pop('Species')

In [16]:
train.shape

(120, 4)

In [17]:
train_y.head()

0    2
1    1
2    2
3    0
4    0
Name: Species, dtype: int64

## Input function

In [28]:
def input_fn(features, labels, training=True, batch_size=256):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features),labels)) # convert inputs to a dataset
    
    if training:
        dataset = dataset.shuffle(1000).repeat() # shuffle and repeat if in training mode
    return dataset.batch(batch_size)

## Feature Columns

In [29]:
# feature col describe how to use the input

my_feature_columns = []
for key in train.keys(): # keys gives us all the count
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


## Building the model

In [30]:
# there are diff models/estimators like DNNClassifier, LinearClassifier
# here we use DNN model, because we dont fine linear coresspondance in our data

classifier = tf.estimator.DNNClassifier(feature_columns=my_feature_columns,
                                       # 2 hidden layers of 30, 10 nodes respectively
                                       hidden_units=[30,10],
                                       # model must choose 3 classes
                                       n_classes=3)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\BHANUT~1\\AppData\\Local\\Temp\\tmp1lts97n9', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Training

In [31]:
classifier.train(input_fn=lambda:input_fn(train, train_y, training=True),
                steps=5000)
# lambda gets rid of make_input_fn function

INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\BHANUT~1\AppData\Local\Temp\tmp1lts97n9\model.ckpt.
INFO:tensorflow:C:\Users\BHANUT~1\AppData\Local\Temp\tmp1lts97n9\model.ckpt-0.data-00000-of-00001
INFO:tensorflow:0
INFO:tensorflow:C:\Users\BHANUT~1\AppData\Local\Temp\tmp1lts97n9\model.ckpt-0.index
INFO:tensorflow:0
INFO:tensorflow:C:\Users\BHANUT~1\AppData\Local\Temp\tmp1lts97n9\model.ckpt-0.meta
INFO:tensorflow:100
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 1.7993027, step = 0
INFO:tensorflow:global_st

In [39]:
eval_result = classifier.evaluate(input_fn=lambda:input_fn(test, test_y, training=False))
print('Test set Accuracy:{accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2023-03-30T21:01:49
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\BHANUT~1\AppData\Local\Temp\tmp1lts97n9\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.18887s
INFO:tensorflow:Finished evaluation at 2023-03-30-21:01:49
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.96666664, average_loss = 0.26681936, global_step = 5000, loss = 0.26681936
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: C:\Users\BHANUT~1\AppData\Local\Temp\tmp1lts97n9\model.ckpt-5000
Test set Accuracy:0.967



## Predictions

In [43]:
# allows to type the features of a flower and see the predected class for it

def input_fn(features, batch_size=256):
    # convert the inputs with out label
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength', 'SepalWidth', 'PetalLength','PetalWidth']
predict={}

print('Enter the numeric values.')
for feature in features:
    valid = True
    while valid:
        val = input(feature+": ")
        if not val.isdigit():valid=False
            
    predict[feature] = [float(val)]
    
predictions = classifier.predict(input_fn=lambda:input_fn(predict))
for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]
    
    print('Prediction is "{}" ({:.1f}%)'.format(SPECIES[class_id],100*probability))

Enter the numeric values.
SepalLength: 8.6
SepalWidth: 5.3
PetalLength: 4.9
PetalWidth: 2.3
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\BHANUT~1\AppData\Local\Temp\tmp1lts97n9\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "Vesicolor" (69.8%)
